# Path Finder

## Setup Spark environment

In [ ]:
%%local
import os
username = os.environ['JUPYTERHUB_USER']
get_ipython().run_cell_magic('configure', line="-f", cell='{ "name":"%s-final-path-finder", "executorMemory":"4G", "executorCores":4, "numExecutors":10, "driverMemory": "4G" }' % username)

In [ ]:
%%send_to_spark -i username -t str -n username

In [ ]:
import pyspark.sql.functions as F
from pyspark.sql.types import *

## Import data from HDFS

In [ ]:
routes = spark.read.csv('/data/sbb/csv/timetable/routes/2019/05/07/routes.csv', header=True )
stops = spark.read.orc('/data/sbb/orc/geostops')

In [ ]:
walking_edges = spark.read.parquet('/user/%s/final/parquet/walking_edges' % username)
transport_edges = spark.read.parquet('/user/%s/final/parquet/transport_edges' % username)

## Shortest paths

In [ ]:
inward_walking_edges = walking_edges.groupby('end_vertex').agg({'start_vertex': 'collect_set'})\
                                                              .withColumnRenamed('collect_set(start_vertex)', 'start_vertices')\
                                                              .toPandas().set_index('end_vertex').to_dict()['start_vertices']

In [ ]:
walking_edge_duration = walking_edges.withColumn('key', F.struct(F.col('start_vertex'), F.col('end_vertex')))\
                                     .toPandas()\
                                     .set_index('key')\
                                     .to_dict()['duration']

In [ ]:
edges = transport_edges.filter('weekday == "monday"')

In [ ]:
# This function assumes a (spark) dataframe with columns 'start_vertex', 'end_vertex', 'start_time', 'duration', 'route_id' 
# edges is a dataframe that represents the edges of the graph, 
# edges should be sorted by order of descending starting time
# inward_walking_edges is a map route_id -> list(route_id), representing the nodes from which we can walk to a given node
# walking_edge_duration is a map (route_id, route_id) -> float, representing the duration of the walk
# end_time should be in the format 'HH:MM'
def latest_departure_paths(edges, end_stop, end_time, inward_walking_edges, walking_edge_duration):
    
    # This function computes whether edge (u, v, t, dur, route_id) can be taken
    # returns update to node u if possible, and None otherwise
    def edge_valid(u, v, time, dur, route_id):
        
        cur_best = next_transfer.get(u)
        next_edge = next_transfer.get(v)
        
        if next_edge: # needs to be defined
            time_v, next_v, route_id_v, dur_v, seq_v = next_edge
            
            if route_id == 'walking':
                if  route_id_v == 'walking' and dur_v + dur <= max_walking_time:
                    time = time_v - dur - transfer_time
                    if (not cur_best) or time > cur_best[0]:
                        return (time, next_v, route_id, dur + dur_v, [u] + seq_v)
                elif route_id_v != 'walking':
                    time = time_v - dur - transfer_time
                    if (not cur_best) or time > cur_best[0]:
                        return (time, v, route_id, dur, [u, v])
                      
            
            elif (route_id == route_id_v or v == end_stop) and time + duration <= time_v: # same route or last_stop
                if (not cur_best) or time > cur_best[0]: # can we improve
                    return (time, next_v, route_id, dur + dur_v, [u] + seq_v)
            
            
            elif time + duration + transfer_time <= time_v: # transfer at node_v
                if (not cur_best) or time > cur_best[0]:
                    return (time, v, route_id, dur, [u, v]) 
        
        # In any other case no update can be made
        return None
                
    def time_to_minutes(timestamp):
        return int(timestamp[:2]) * 60 + int(timestamp[3:5])
    
    transfer_time = 2 # at least 2 minutes to transfer
    max_walking_time = 10

    next_transfer = {} # stores for each node u the node v 
                       # where the next transfer happens format (departure_time, v, route_id to v, duration)
    all_roads = {}
    end_time = time_to_minutes(end_time)
    start_time = end_time - 120 # look only at edges departing at most 2 hours before end_time
    
    next_transfer[end_stop] = (end_time, end_stop, None, 0, [end_stop])
    all_roads[end_stop] = [(end_time, end_stop, None, 0, [end_stop])]

    edges = edges.filter(F.col('start_time').between(start_time, end_time)).toPandas().sort_values(by=['start_time'], ascending=False).to_numpy()

    for row in edges:
        
        start_vertex, end_vertex, start, duration, route_id = row[:5]
        
        update = edge_valid(start_vertex, end_vertex, start, duration, route_id)
        if update:
            if(all_roads.get(start_vertex) != None):    
                all_roads[start_vertex].append(update)
            elif (all_roads.get(start_vertex) == None):
                all_roads[start_vertex] = [update]
                
            next_transfer[start_vertex] = update
                    
            w_start_vertex = start_vertex
            for w_end_vertex in inward_walking_edges.get(w_start_vertex, []):
                duration = walking_edge_duration[(w_start_vertex, w_end_vertex)]
                w_start = start 
                update = edge_valid(w_start_vertex, w_end_vertex, w_start, duration, 'walking')
                if update:
                    if(all_roads.get(w_start_vertex) != None):
                        all_roads[w_start_vertex].append(update)
                    elif (all_roads.get(start_vertex) == None):
                        all_roads[w_start_vertex] = [update]
                    
                    next_transfer[w_start_vertex] = update

    return all_roads

In [ ]:
import time
start = time.time()
all_roads = latest_departure_paths(edges, '8591221', '20:00', inward_walking_edges, walking_edge_duration)
end = time.time() 
print(end - start)

In [ ]:
# This cell is for generating best k paths
def select_next(time, paths_from_next_stop):
    rev = paths_from_next_stop[::-1]
    
    for i in rev:
        if (i[0] >= time):
            return i
    
    return (None, None, None, None , [])

def generate_best_k_paths(k_paths, start_stop, k):
    l = k_paths[start_stop]
    nbre_paths = k
    
    if (l == None):
        return None
    
    if(len(l) < k):
        nbre_paths = len(l)
    
    possible = l[::-1][0:nbre_paths]
    
    roads = []
    for p in possible:
        start_stop_ = start_stop
        time, next_stop, route_id, duration, stops = p
        journey = []
        i = 1
        while next_stop != start_stop_:
            journey.append(
                (i, start_stop_, next_stop, route_id, time, duration, stops)
            )
            
            start_stop_ = next_stop
            i += 1
            
            paths_from_next_stop = k_paths[next_stop]
            m = select_next(time + duration, paths_from_next_stop)
            
            if (m[0] != None):
                time, next_stop, route_id, duration, stops = m
        
        roads.append(journey)
        
    return roads

In [ ]:
best_k_paths = generate_best_k_paths(all_roads, '8591250', 4 )

In [ ]:
def journey_to_df(journey_list):
    arr = []
    journey_number = 0
    for journey in journey_list:
        journey_number += 1
        arr = arr + [(journey_number,) + seg for seg in journey]
    
    schema = StructType([
        StructField("journey_number", IntegerType(), True),
        StructField("segment_number", IntegerType(), True),
        StructField("start_vertex", StringType(), True),
        StructField("end_vertex", StringType(), True),
        StructField("route_id", StringType(), True),
        StructField("departure_time", StringType(), True),
        StructField("duration", StringType(), True),
        StructField("stop_seq", ArrayType(StringType()), True)
    ])
    
    journey_df = spark.createDataFrame(arr, schema)
    
    return journey_df

In [ ]:
journey_df = journey_to_df(best_k_paths)

In [ ]:
journey_df.show()

## Send to local

In [ ]:
%%spark -o journey_df -n -1

In [ ]:
%%local
journey_df['start_vertex'] = journey_df.start_vertex.astype(str)
journey_df['end_vertex'] = journey_df.end_vertex.astype(str)

In [ ]:
%%spark -o stop_id_to_stop -n -1
stop_id_to_stop = stops.select('stop_id', 'stop_name')

In [ ]:
%%spark -o stop_id_to_lon -n -1
stop_id_to_lon = stops.select('stop_id', 'stop_lon')

In [ ]:
%%spark -o stop_id_to_lat -n -1
stop_id_to_lat = stops.select('stop_id', 'stop_lat')

In [ ]:
%%spark -o route_id_to_route_name -n -1
route_id_to_route_name = routes.withColumn('route_name', F.concat(F.col('route_desc'), F.lit(' '), F.col('route_short_name')))\
                               .select('route_id', 'route_desc', 'route_short_name', 'route_name')

## Display results

In [ ]:
%%local
stop_id_to_stop_dict = stop_id_to_stop.set_index('stop_id').to_dict()['stop_name']

stop_id_to_lon_dict = stop_id_to_lon.set_index('stop_id').to_dict()['stop_lon']
stop_id_to_lat_dict = stop_id_to_lat.set_index('stop_id').to_dict()['stop_lat']

route_id_to_route_name_dict = route_id_to_route_name.set_index('route_id').to_dict()['route_name']
route_id_to_route_name_dict['walking'] = 'Walk'

In [ ]:
%%local
def minutes_to_timestamp(minutes):
    if minutes < 60:
        return str(minutes)
    return str(minutes // 60) + ':' + str(minutes % 60).zfill(2)

### Print paths

In [ ]:
%%local
def pretty_print_journey(journey):
    journey_str = '\n'.join(["%s : %s\n|\n| %s (%s min)\n| %s\n|\n%s : %s\n" % (minutes_to_timestamp(time), 
                                                                             stop_id_to_stop_dict[start],
                                                                             route_id_to_route_name_dict[route_id],
                                                                             minutes_to_timestamp(duration),
                                                                             ' -> '.join([stop_id_to_stop_dict[s] for s in stops]) if route_id != 'walking' else '',
                                                                             minutes_to_timestamp(time + duration),
                                                                             stop_id_to_stop_dict[end]) for _, start, end, route_id, time, duration, stops in journey])
    return journey_str

In [ ]:
%%local
def journeys_to_str(df):
    if len(df) == 0:
        return "No possible path"
    
    n = df['journey_number'].unique()

    res = []
    for i in sorted(n):
        segment_list = sorted(df[df.journey_number == i].values[:,1:], key = lambda e: e[0])
        res.append(pretty_print_journey(segment_list))
    return '\n******************************\n'.join(res)

In [ ]:
%%local
print(journeys_to_str(journey_df))

### Map visualization

In [ ]:
%%local
import matplotlib.pyplot as plt

def get_cmap(n, name='hsv'):
    '''Returns a function that maps each index in 0, 1, ..., n-1 to a distinct 
    RGB color; the keyword argument name must be a standard mpl colormap name.'''
    return plt.cm.get_cmap(name, n)

In [ ]:
%%local
import plotly.graph_objects as go
from matplotlib.colors import rgb2hex
from ipywidgets import interact

def map_plot():
    fig = go.Figure()

    fig.add_trace(go.Scattermapbox(
    ))

    fig.update_layout(
        autosize=True,
        hovermode='closest',
        mapbox=dict(
            style='carto-positron',
            # style='open-street-map',
            bearing=0,
            center=dict(
                lat=47.378177,
                lon=8.540192
            ),
            pitch=0,
            zoom=12
        ),
        height=900,
        width=1200
    )

    return fig

In [ ]:
%%local

fig = map_plot()
stream_fig = go.FigureWidget(fig)
stream_data = stream_fig.data

@interact(path_number=journey_df.journey_number.unique())
def show_path(path_number):
    df=journey_df

    if len(df) == 0:
        return

    n = df['journey_number'].unique()

    cmap = get_cmap(max(n)+1)
    color = rgb2hex(cmap(path_number-1))

    journey = sorted(df[df.journey_number == path_number].values[:,1:], key = lambda e: e[0])        
        
    stops_per_segment = [seg[-1] for seg in journey]
    
    stops_per_segment_no_duplicates = [stops_per_segment[0]] + [ss[1:] for ss in stops_per_segment[1:]]

    stops = [item for sublist in stops_per_segment_no_duplicates for item in sublist]
    
    lats = [stop_id_to_lat_dict[s] for s in stops]
    lons = [stop_id_to_lon_dict[s] for s in stops]
    
    with stream_fig.batch_update():
        stream_fig.data = stream_fig.data[:1]
        stream_fig.update_layout(mapbox_center=dict(lat= sum(lats)/len(lats), lon= sum(lons)/len(lons)))
    
    
    stops_per_route = [(seg[3], seg[-1]) for seg in journey]
    
    for route_id, stops in stops_per_route:
        stop_names = [stop_id_to_stop_dict[s] for s in stops]
        with stream_fig.batch_update():
            stream_fig.add_trace(go.Scattermapbox(
                lat=[stop_id_to_lat_dict[s] for s in stops],
                lon=[stop_id_to_lon_dict[s] for s in stops],
                mode='lines+markers',
                marker=go.scattermapbox.Marker(
                    size=6,
                ),
                opacity=1,
                hovertext=stop_names,
                name=route_id_to_route_name_dict[route_id],
                hoverinfo="text+name"
            ))
        
stream_fig